In [6]:
import timm, torch

model_name = "pvt_v2_b2_li"
# model_name ="pvt_v2_b1.pth"
# model_name ="pvt_v2_b0.pth"
# model_name ="pvt_v2_b2.pth"
# model_name ="pvt_v2_b3.pth"

model_timm = timm.create_model(model_name, pretrained=True)
state_dict = model_timm.state_dict()

for fpn in [True, False]:
    for c4 in [True, False]:
        file_name = f"{model_name}" if not fpn else f"{model_name}_fpn"
        file_name = f"{file_name}" if not c4 else f"{file_name}_c4"

        new_model_dict = {}
        for key, value in state_dict.items():
            if 'stages.3' in key and c4:
                new_key = key.replace('stages.3', 'roi_heads.box_head.stage')
            else:
                if fpn:
                    new_key = f"backbone.bottom_up.{key}"
                else:
                    new_key= f"backbone.{key}"
            new_model_dict[new_key] = value
        torch.save(new_model_dict, f"weights/{file_name}.pth")


In [21]:
import timm, torch

model_name = "twins_svt_small"
model = timm.create_model(model_name, pretrained=True)
state_dict = model.state_dict()

# Define the prefixes to be prepended
prefix_map = {
    "blocks.3",
    "pos_block.3",
    "patch_embeds.3",
}

for fpn in [True, False]:
    for c4 in [True, False]:
        file_name = f"{model_name}" if not fpn else f"{model_name}_fpn"
        file_name = f"{file_name}" if not c4 else f"{file_name}_c4"

        new_model_dict = {}
        for key, value in state_dict.items():
            prefix = "backbone"

            if fpn:
                prefix = f"{prefix}.bottom_up"

            if c4:
                for stage in prefix_map:
                    if stage in key or key.startswith("norm"):
                        prefix = "roi_heads.box_head"

            new_key = f"{prefix}.{key}"
            new_model_dict[new_key] = value
        torch.save(new_model_dict, f"weights/{file_name}.pth")